In [2]:
# import sys
# import os
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [3]:
import os
os.environ["NUMBA_DEBUG_CACHE"] = "1"

In [4]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_enums import DynamicOrderSettingsArrays
from nb_quantfreedom.utils import pretty_qf
from requests import get
from numba import njit, types, typed
from numba.experimental import jitclass
import logging
from nb_quantfreedom.nb_enums import *
from nb_quantfreedom.nb_helper_funcs import dos_cart_product
from nb_quantfreedom.strategies.nb_strategy import (
    strat_bt_create_ind,
    strat_evaluate,
    strat_get_current_ind_settings,
    strat_get_ind_set_str,
    strat_get_total_ind_settings,
)
from nb_quantfreedom.nb_simulate import sim_df_backtest
from nb_quantfreedom.sim_order_records import sim_get_or


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [5]:
candles_np = mufex_main.get_candles_df(symbol="BTCUSDT", timeframe="5m", candles_to_dl=2000)

It took 00 mins and 01 seconds to download 2000 candles


In [6]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings(
    divide_records_array_size_by=15,
    gains_pct_filter=-90,
    total_trade_filter=60,
    upside_filter=-np.inf,
)
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([6, 9, 12]),
    max_trades=np.array([5]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([1, 3, 6]),
    sl_based_on_add_pct=np.array([0.01, 0.03, .06]),
    sl_based_on_lookback=np.array([10, 30]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    sl_to_be_ze_type=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([.1, 0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2, 4]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = dos_cart_product(
    dos_arrays=dos_arrays,
)

In [7]:
sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
)

[cache] index loaded from 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_custom_logger.logger_pass-215.py310.nbi'
[cache] data loaded from 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_custom_logger.logger_pass-215.py310.1.nbc'
[cache] index loaded from 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_custom_logger.stringer_pass-220.py310.nbi'
[cache] data loaded from 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_custom_logger.stringer_pass-220.py310.1.nbc'
[cache] index loaded from 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\strategies\\__pycache__\\nb_strategy.strat_get_total_ind_settings-96.py310.nbi'
[cache] data loaded from 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\strategies\\__pycache__\\nb_strategy.strat_get_total_ind_settings-96.py310.1.nbc'
Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to 

,ind_set_idx,dos_index,total_trades,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,54,87.0,-83.63,18.39,0.0,691.951,-836.2941,163.7059
1,0,60,70.0,-85.87,15.71,0.0,536.800,-858.7350,141.2650
2,0,72,75.0,-72.26,22.67,0.0,706.132,-722.5769,277.4231
3,0,75,71.0,-79.06,21.13,0.0,601.102,-790.6283,209.3717
4,0,78,64.0,-83.63,14.06,0.0,474.199,-836.3196,163.6804
...,...,...,...,...,...,...,...,...,...
110,3,451,64.0,-82.96,14.06,0.0,1006.202,-829.6426,170.3574
111,3,453,73.0,-56.57,23.29,0.0,1778.896,-565.7242,434.2758
112,3,454,64.0,-76.43,15.62,0.0,952.873,-764.2585,235.7415
113,3,468,73.0,-38.38,27.40,0.0,2270.176,-383.8328,616.1672


In [8]:
order_records_df = sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=0,
    dos_index=0,
)

[cache] index loaded from 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_helper_funcs.fill_order_records-255.py310.nbi'
[cache] data loaded from 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_helper_funcs.fill_order_records-255.py310.1.nbc'
[cache] index loaded from 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_helper_funcs.fill_order_records-255.py310.nbi'
[cache] data loaded from 'E:\\Coding\\backtesters\\QuantFreedom\\nb_quantfreedom\\__pycache__\\nb_helper_funcs.fill_order_records-255.py310.2.nbc'


In [9]:
order_records_df.T

,0,1,2,3,4,5,6,7,8,9,...,213,214,215,216,217,218,219,220,221,222
ind_set_idx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
or_set_idx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bar_idx,100,100,101,102,108,109,109,110,119,120,...,1725,1726,1909,1910,1911,1912,1913,1924,1925,1926
timestamp,1697697000000,1697697000000,1697697300000,1697697600000,1697699400000,1697699700000,1697699700000,1697700000000,1697702700000,1697703000000,...,1698184500000,1698184800000,1698239700000,1698240000000,1698240300000,1698240600000,1698240900000,1698244200000,1698244500000,1698244800000
datetime,2023-10-19 06:30:00,2023-10-19 06:30:00,2023-10-19 06:35:00,2023-10-19 06:40:00,2023-10-19 07:10:00,2023-10-19 07:15:00,2023-10-19 07:15:00,2023-10-19 07:20:00,2023-10-19 08:05:00,2023-10-19 08:10:00,...,2023-10-24 21:55:00,2023-10-24 22:00:00,2023-10-25 13:15:00,2023-10-25 13:20:00,2023-10-25 13:25:00,2023-10-25 13:30:00,2023-10-25 13:35:00,2023-10-25 14:30:00,2023-10-25 14:35:00,2023-10-25 14:40:00
order_status,EntryFilled,StopLossFilled,EntryFilled,EntryFilled,StopLossFilled,EntryFilled,StopLossFilled,EntryFilled,TakeProfitFilled,EntryFilled,...,EntryFilled,EntryFilled,MovedTSL,MovedTSL,StopLossFilled,EntryFilled,EntryFilled,TakeProfitFilled,EntryFilled,EntryFilled
equity,1000.0,970.0086,970.0086,970.0086,910.6662,910.6662,883.6676,883.6676,864.0165,864.0165,...,35.8332,35.8332,35.8332,35.8332,37.0864,37.0864,37.0864,35.1445,35.1445,35.1445
available_balance,865.089,970.0086,860.662,736.585,910.6662,777.976,883.6676,742.258,864.0165,803.293,...,34.409,33.026,33.026,33.026,37.0864,35.011,32.425,35.1445,33.54,31.842
cash_borrowed,11555.86,NaN,9366.064,19884.567,NaN,11365.554,NaN,12112.462,NaN,4246.116,...,51.498,99.945,99.945,99.945,NaN,131.351,299.96,NaN,73.647,151.472
cash_used,134.911,NaN,109.346,233.423,NaN,132.69,NaN,141.41,NaN,60.723,...,1.424,2.807,2.807,2.807,NaN,2.075,4.661,NaN,1.605,3.303


In [10]:
list(order_records_df.columns)

['ind_set_idx',
 'or_set_idx',
 'bar_idx',
 'timestamp',
 'datetime',
 'order_status',
 'equity',
 'available_balance',
 'cash_borrowed',
 'cash_used',
 'average_entry',
 'fees_paid',
 'leverage',
 'liq_price',
 'possible_loss',
 'total_trades',
 'entry_size_asset',
 'entry_size_usd',
 'entry_price',
 'exit_price',
 'position_size_asset',
 'position_size_usd',
 'realized_pnl',
 'sl_pct',
 'sl_price',
 'tp_pct',
 'tp_price']

In [30]:
temp_df = order_records_df[
    ["order_status", "timestamp", "average_entry", "entry_price", "liq_price", "sl_price", "tp_price"]
]

In [31]:
entries_df = temp_df[temp_df['order_status']=='EntryFilled']

In [32]:
entries_list_df = entries_df.values[:,1:].tolist()

In [33]:
entries_list = candles_np[:,0].tolist()

In [38]:
candles_np.tolist()

[[1697667000000.0, 28281.1, 28315.9, 28279.6, 28298.6],
 [1697667300000.0, 28298.6, 28305.6, 28287.5, 28303.2],
 [1697667600000.0, 28303.2, 28324.0, 28300.5, 28303.3],
 [1697667900000.0, 28303.3, 28317.5, 28293.3, 28296.1],
 [1697668200000.0, 28296.1, 28300.0, 28295.9, 28299.7],
 [1697668500000.0, 28299.7, 28299.7, 28285.5, 28292.7],
 [1697668800000.0, 28292.7, 28299.9, 28292.7, 28294.0],
 [1697669100000.0, 28294.0, 28315.1, 28288.3, 28306.2],
 [1697669400000.0, 28306.2, 28322.7, 28305.9, 28315.6],
 [1697669700000.0, 28315.6, 28330.2, 28313.9, 28329.9],
 [1697670000000.0, 28329.9, 28360.5, 28325.1, 28333.2],
 [1697670300000.0, 28333.2, 28337.0, 28309.1, 28313.5],
 [1697670600000.0, 28313.5, 28319.1, 28308.8, 28314.7],
 [1697670900000.0, 28314.7, 28314.8, 28290.1, 28293.8],
 [1697671200000.0, 28293.8, 28308.6, 28293.8, 28308.6],
 [1697671500000.0, 28308.6, 28318.9, 28303.0, 28318.9],
 [1697671800000.0, 28318.9, 28345.5, 28295.1, 28300.7],
 [1697672100000.0, 28300.7, 28303.0, 28292.3, 28

In [35]:
final_list = []
for value in entries_list:
    if value != entries_list_df[0][0]:
        final_list.append([value])
    else:
        final_list.append(entries_list_df[0])
        del entries_list_df[0]

IndexError: list index out of range

In [40]:
data = {
    "entries": final_list,
    "candles": candles_np.tolist(),
}